# Summary

This notebook demonstrates how to use the dot matrix module on the Raspberry Pi to create a pulsing circle animation. The following libraries are required:
```
spidev
luma.led-matrix
```

Follow the instructions [here](https://docs.sunfounder.com/projects/raphael-kit/en/latest/python_pi5/pi5_1.1.6_led_dot_matrix_python.html) to hook up the hardware and install the libraries.

In [13]:
import math
import time

from luma.core.interface.serial import noop, spi
from luma.core.legacy import text
from luma.core.legacy.font import CP437_FONT, LCD_FONT, proportional
from luma.core.render import canvas
from luma.core.virtual import viewport
from luma.led_matrix.device import max7219

# Basic Shapes

In [14]:
# Initialize the SPI interface and the MAX7219 device
serial = spi(port=0, device=0, gpio=noop())
device = max7219(serial, rotate=0, width=8, height=8)

# 8x8 matrix
device.bounding_box

(0, 0, 7, 7)

### A hollow half circle

In [4]:
with canvas(device) as draw:
    draw.arc(device.bounding_box, start=0, end=180, fill="white", width=1)

### A filled in half-circle

In [5]:
with canvas(device) as draw:
    draw.chord(device.bounding_box, start=180, end=360, fill="white", outline=None, width=1)

### A circle centered in the matrix

In [6]:
center = (3.5, 3.5)  # Center of the 8x8 matrix

# Each time you want to redraw the circle, you have to do a new canvas
with canvas(device) as draw:
    draw.circle(center, radius=4, fill="black", outline="white", width=1)

### Text moving across the matrix

In [7]:
virtual = viewport(device, width=200, height=400)

with canvas(virtual) as draw:
    text(draw, (0, 0), "Hello, friend!", fill="white", font=proportional(CP437_FONT))

for offset in range(150):
    virtual.set_position((offset, 0))
    time.sleep(0.1)

# Pulsing Circle Animation

In [8]:
serial = spi(port=0, device=0, gpio=noop())
device = max7219(serial, cascaded=1)


def draw_circle():
    with canvas(device) as draw:
        draw.ellipse(device.bounding_box, outline="white", fill="white")


def asymmetric_wave(t):
    """
    Very fast rise, slow decay.
    - t in [0, 1]
    - Immediate full brightness for first 10% of cycle
    - Smooth exponential decay afterward
    """
    if t < 0.1:
        return 1.0
    else:
        # Decay from 1 → ~0 over rest of cycle
        decay = math.exp(-5 * (t - 0.1))
        return max(0.0, decay)


def pulse_circle(duration=5.0, fps=30, min_brightness=4, max_brightness=255):
    """
    Pulses a circle with fast rise and slow fade.
    """
    cycle_time = 1.0
    start_time = time.time()

    while time.time() - start_time < duration:
        t = (time.time() - start_time) % cycle_time
        norm_t = t / cycle_time
        brightness_factor = asymmetric_wave(norm_t)
        brightness = min_brightness + int(
            brightness_factor * (max_brightness - min_brightness)
        )
        device.contrast(brightness)
        draw_circle()
        time.sleep(1 / fps)


pulse_circle(duration=5.0, fps=60)

In [12]:
serial = spi(port=0, device=0, gpio=noop())
device = max7219(serial, cascaded=1)


def draw_heart():
    pixels = [
        # fmt: off
                (1, 1),                                 (6, 1),
        (0, 2), (1, 2), (2, 2),                 (5, 2), (6, 2), (7, 2),
        (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (7, 3),
                (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4),
                        (2, 5), (3, 5), (4, 5), (5, 5),
                                (3, 6), (4, 6)
        # fmt: on
    ]
    with canvas(device) as draw:
        for x, y in pixels:
            draw.point((x, y), fill="white")


def pulse_heart(
    duration=5.0,
    fps=30,
    cycle_time=1.0,
    min_brightness=4,
    max_brightness=255,
):
    start_time = time.time()

    while time.time() - start_time < duration:
        t = (time.time() - start_time) % cycle_time
        norm_t = t / cycle_time
        if norm_t < 0.1:
            brightness_factor = 1.0  # instant pulse
        else:
            brightness_factor = max(0.0, math.exp(-5 * (norm_t - 0.1)))
        brightness = min_brightness + int(
            brightness_factor * (max_brightness - min_brightness)
        )
        device.contrast(brightness)
        draw_heart()
        time.sleep(1 / fps)


pulse_heart(
    duration=5.0,
    fps=30,
    cycle_time=0.5,
    min_brightness=0,
    max_brightness=255,
)

# Ripple Animation

In [10]:
# Predefined ripple frames (bounding boxes for ellipses)
ripple_frames = [
    [(3, 3, 4, 4)],  # Center dot
    [(2, 2, 5, 5)],  # First ring
    [(1, 1, 6, 6)],  # Second ring
    [(0, 0, 7, 7)],  # Full frame
]


def ripple_animation(duration=5.0, fps=4):
    """
    Displays a ripple animation expanding from the center.
    duration: total time to run in seconds
    fps: frames per second
    """
    start_time = time.time()
    frame_duration = 1 / fps

    while time.time() - start_time < duration:
        for frame in ripple_frames:
            with canvas(device) as draw:
                for bbox in frame:
                    draw.ellipse(bbox, outline="white")
            time.sleep(frame_duration)


ripple_animation(duration=5.0)

# Voice activity animation

In [11]:
serial = spi(port=0, device=0, gpio=noop())
device = max7219(serial, cascaded=1)


def voice_bounce_bars(duration=5.0, fps=15, speed=2.5):
    """
    Draws 5 vertical bars that bounce up/down like a voice activity animation.
    """
    start_time = time.time()

    bar_columns = [0, 2, 3, 5, 7]  # spread across 8x8
    bar_offsets = [0.0, 0.3, 0.6, 0.3, 0.0]  # phase offset per bar

    while time.time() - start_time < duration:
        t = time.time() - start_time
        with canvas(device) as draw:
            for i, x in enumerate(bar_columns):
                phase = bar_offsets[i]
                # Height will range from 2 to 6 (for visual bounce)
                height = 2 + int(2.5 * (math.sin(speed * t + phase * math.pi) + 1))
                for y in range(8 - height, 8):
                    draw.point((x, y), fill="white")
        time.sleep(1 / fps)


voice_bounce_bars(duration=5, fps=60, speed=10.0)